# Train/Test/Validation Split

In [1]:
from dsm import datasets
from dsm.dsm_api import DeepSurvivalMachines

import numpy as np

from sklearn.model_selection import ParameterGrid
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc

### Loading the Support Dataset 

In [2]:
x, t, e = datasets.load_dataset('SUPPORT')

### Computing the required quantile of the data 

In [3]:
times = np.quantile(t[e == 1], [0.25, 0.5, 0.75]).tolist()

### Defining the train, test and validation size

In [4]:
data_size = len(x)
tr_size = round(data_size * 0.70)
vl_size = round(data_size * 0.10)
te_size = round(data_size * 0.20)

### Splitting the data into train, test and validation sets

In [8]:
x_train = x[: tr_size]
t_train = t[: tr_size]
e_train = e[: tr_size]

x_test = x[-te_size :]
t_test = t[-te_size :]
e_test = e[-te_size :]

x_val = x[tr_size : tr_size + vl_size]
t_val = t[tr_size : tr_size + vl_size]
e_val = e[tr_size : tr_size + vl_size]

In [9]:
et_train = np.array([(e_train[i], t_train[i]) for i in range(len(e_train))],
                 dtype = [('e', bool), ('t', int)])
et_test = np.array([(e_test[i], t_test[i]) for i in range(len(e_test))],
                 dtype = [('e', bool), ('t', int)])
et_val = np.array([(e_val[i], t_val[i]) for i in range(len(e_val))],
                 dtype = [('e', bool), ('t', int)])

In [10]:
print(et_train.shape, et_test.shape, et_val.shape)

(6374,) (1821,) (910,)


### Defining the parameter grid

In [11]:
param_grid = {'k' : [3, 4, 6],
              'distribution' : ['LogNormal', 'Weibull'],
              'learning_rate' : [ float(1e-4), float(1e-3)],
              'layers' : [ [], [100], [100, 100] ]
             }
params = ParameterGrid(param_grid)

### Validation 

In [12]:
for param in params:
    model = DeepSurvivalMachines(k = param['k'],
                                 distribution = param['distribution'],
                                 layers = param[ 'layers'])
    model.fit(x_train, t_train, e_train, learning_rate = param['learning_rate'])
    out_risk = model.predict_risk(x_val, times)
    out_survival = model.predict_survival(x_val, times)

    cis = []
    brs = []
    for i, _ in enumerate(times):
        cis.append(concordance_index_ipcw(et_train, et_val, out_risk[:, i], times[i])[0])

    brs.append(brier_score(et_train, et_val, out_survival, times)[1])
    print("Concordance Index:", np.mean(cis))
    print("Brier Score:", np.mean(brs))
    cdauc = []
    for i, _ in enumerate(times):
        cdauc.append(cumulative_dynamic_auc(et_train, et_val, out_risk[:, i], times[i])[0])
    print("Cumulative_dynamic AUC ", np.mean(cdauc))
    print(param)
    #scores  = scores + [[grid, np.mean(cis), np.mean(brs), np.mean(cdauc)]]

  0%|          | 0/10000 [00:00<?, ?it/s]/home/kishan/Documents/Projects/deepsurvival/deep_survival/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
  0%|          | 46/10000 [00:00<00:22, 452.18it/s]

Concordance Index: 0.45340340953800856
Brier Score: 0.2120856405467805
Cumulative_dynamic AUC  0.4325276050509048
{'distribution': 'LogNormal', 'k': 3, 'layers': [], 'learning_rate': 0.0001}


  1%|          | 61/10000 [00:00<00:16, 602.23it/s]

Concordance Index: 0.5868530920885563
Brier Score: 0.2000913347679684
Cumulative_dynamic AUC  0.6063103268431748
{'distribution': 'LogNormal', 'k': 3, 'layers': [], 'learning_rate': 0.001}


  1%|          | 58/10000 [00:00<00:17, 572.61it/s]

...

Concordance Index: 0.7085613962141747
Brier Score: 0.18772750978036887
Cumulative_dynamic AUC  0.7349156454958966
{'distribution': 'Weibull', 'k': 6, 'layers': [100], 'learning_rate': 0.001}


  1%|          | 65/10000 [00:00<00:15, 646.28it/s]

Concordance Index: 0.6568985178048318
Brier Score: 0.202961664621087
Cumulative_dynamic AUC  0.6705386354241947
{'distribution': 'Weibull', 'k': 6, 'layers': [100, 100], 'learning_rate': 0.0001}


100%|██████████| 1/1 [00:00<00:00,  4.08it/s]

Concordance Index: 0.6995013567080287
Brier Score: 0.18538782612760984
Cumulative_dynamic AUC  0.7319456134570768
{'distribution': 'Weibull', 'k': 6, 'layers': [100, 100], 'learning_rate': 0.001}


### Training  

In [13]:
model = DeepSurvivalMachines(k = 3, distribution = 'Weibull', layers = [100])
model.fit(x_train, t_train, e_train, learning_rate = float(1e-3))

100%|██████████| 1/1 [00:00<00:00,  7.03it/s]


### Prediction 

In [14]:
out_risk = model.predict_risk(x_test, times)
out_survival = model.predict_survival(x_test, times)

### Evaluation

In [15]:
cis = []
brs = []
for i, _ in enumerate(times):
    cis.append(concordance_index_ipcw(et_train, et_test, out_risk[:, i], times[i])[0])

brs.append(brier_score(et_train, et_test, out_survival, times)[1])
print("Concordance Index:", np.mean(cis))
print("Brier Score:", np.mean(brs))
cdauc = []
for i, _ in enumerate(times):
    cdauc.append(cumulative_dynamic_auc(et_train, et_test, out_risk[:, i], times[i])[0])
print("Cumulative_dynamic AUC ", np.mean(cdauc))

Concordance Index: 0.7023360857164943
Brier Score: 0.17641956933412173
Cumulative_dynamic AUC  0.7260571933183376
